In [7]:
# Importing Required Libraries
import os
import librosa
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from keras.models import Sequential


In [8]:
def extract_feature(file_name):
    try:
        audio_data, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
        mfccs = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=40)
        mfccs_processed = np.mean(mfccs.T, axis=0)
        return mfccs_processed
    except Exception as e:
        print(f"Error encountered while parsing file: {file_name}")
        print(e)
        return None

    

In [25]:
if __name__ == '__main__':
    # Define the path to the dataset
    data_path = "C:/Users/ADMIN/Downloads/ML_project/genres_original"

    # Define the number of MFCCs to extract
    num_mfcc = 40

    # Define the number of audio files to read per genre
    num_files_per_genre = 100

    # Define the list of genres to use
    genres = ["blues", "classical", "country", "disco", "hiphop", "jazz", "metal", "pop", "reggae", "rock"]

    # Initialize empty arrays to store features and labels
    features = None
    labels = None

    # Loop through each genre
    for genre in genres:
        # Define the path to the genre folder
        genre_path = os.path.join(data_path, genre)

        # Loop through the first N audio files in the genre folder
        for i, file_name in enumerate(os.listdir(genre_path)[:num_files_per_genre]):
            # Construct the full path to the audio file
            file_path = os.path.join(genre_path, file_name)

            # Extract the MFCCs for the audio file
            mfccs = extract_feature(file_path)

            # If the MFCCs were successfully extracted, add them to the feature matrix and label vector
            if mfccs is not None:
                if features is None:
                    features = mfccs.reshape(1, -1)
                    labels = np.array([genre])
                else:
                    features = np.vstack([features, mfccs.reshape(1, -1)])
                    labels = np.append(labels, genre)
    # Encoding Labels to Integers
    le = LabelEncoder()
    labels = le.fit_transform(labels)

    # Splitting data into training and testing
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2)

    # Initialize the neural network model
    model = Sequential()

    # Add layers to the model
    model.add(Dense(units=128, activation='elu', input_shape=(num_mfcc,)))
    model.add(Dense(units=64, activation='relu'))
    model.add(Dense(units=32, activation='elu'))
    model.add(Dense(units=len(genres), activation='softmax'))
    
    # Compiling the Model
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    # Training the Model on the Training Data
    model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        batch_size=32,
        epochs=100
    )

    # Evaluating the Model on the Testing Data
    loss, accuracy = model.evaluate(X_test, y_test)
    print("Accuracy:", accuracy)


Epoch 1/100
25/25 [==============================] - 1s 8ms/step - loss: 7.2297 - accuracy: 0.1587 - val_loss: 2.4250 - val_accuracy: 0.2900
Epoch 2/100
25/25 [==============================] - 0s 4ms/step - loss: 2.1691 - accuracy: 0.3363 - val_loss: 1.8219 - val_accuracy: 0.4400
Epoch 3/100
25/25 [==============================] - 0s 3ms/step - loss: 1.7301 - accuracy: 0.4175 - val_loss: 1.7132 - val_accuracy: 0.4450
Epoch 4/100
25/25 [==============================] - 0s 4ms/step - loss: 1.5383 - accuracy: 0.4925 - val_loss: 1.5556 - val_accuracy: 0.4900
Epoch 5/100
25/25 [==============================] - 0s 4ms/step - loss: 1.3760 - accuracy: 0.5225 - val_loss: 1.4707 - val_accuracy: 0.5050
Epoch 6/100
25/25 [==============================] - 0s 4ms/step - loss: 1.3024 - accuracy: 0.5375 - val_loss: 1.5167 - val_accuracy: 0.4900
Epoch 7/100
25/25 [==============================] - 0s 4ms/step - loss: 1.3060 - accuracy: 0.5612 - val_loss: 1.4990 - val_accuracy: 0.4950
Epoch 8/100
2

Epoch 59/100
25/25 [==============================] - 0s 3ms/step - loss: 0.0935 - accuracy: 0.9837 - val_loss: 1.7807 - val_accuracy: 0.6700
Epoch 60/100
25/25 [==============================] - 0s 3ms/step - loss: 0.0989 - accuracy: 0.9812 - val_loss: 1.8039 - val_accuracy: 0.6600
Epoch 61/100
25/25 [==============================] - 0s 3ms/step - loss: 0.0875 - accuracy: 0.9787 - val_loss: 1.9512 - val_accuracy: 0.6200
Epoch 62/100
25/25 [==============================] - 0s 4ms/step - loss: 0.0866 - accuracy: 0.9862 - val_loss: 1.9261 - val_accuracy: 0.6250
Epoch 63/100
25/25 [==============================] - 0s 3ms/step - loss: 0.0985 - accuracy: 0.9712 - val_loss: 2.1159 - val_accuracy: 0.6350
Epoch 64/100
25/25 [==============================] - 0s 3ms/step - loss: 0.1055 - accuracy: 0.9700 - val_loss: 1.9670 - val_accuracy: 0.6600
Epoch 65/100
25/25 [==============================] - 0s 3ms/step - loss: 0.0829 - accuracy: 0.9825 - val_loss: 1.9536 - val_accuracy: 0.6450
Epoch 

In [21]:
# Save the trained model
model.save('my_model.h5')

# Load the saved model
loaded_model = keras.models.load_model('my_model.h5')

# Predict the genre of an audio file
mfccs = extract_feature('C:/Users/ADMIN/Downloads/ML_project/audio_file.wav')

input_data = mfccs.reshape(1, -1)
# print(input_data)
predicted_genre = loaded_model.predict(input_data)

# Extract genre name from probabilities
genres = ["blues", "classical", "country", "disco", "hiphop", "jazz", "metal", "pop", "reggae", "rock"]
predicted_genre_index = np.argmax(predicted_genre)
predicted_genre_name = genres[predicted_genre_index]
print('Predicted genre: ', predicted_genre_name)


1/1 [==============================] - 0s 48ms/step
Predicted genre:  rock
